<div style="text-align: center"> <h1> Final Project Report </h1> </div>

<div style="text-align: right"> Luke Burford and Drew Boston </div>

## Introduction ##

This notebook details our journey to train a neural network to complete the first level of [Super Mario Bros](https://en.wikipedia.org/wiki/Super_Mario_Bros), a sidescrolling game released for the Nintendo Entertainment System in 1985. ![title](SMB.png)

Originally, we had determined to start from scratch; we would first find a suitable emulator, create the network and set up the training process, and continue training until we had a suitable network. This proved, however, to be a far more difficult endeavor than we had expected. 

We had some neural network resources available from a class project (specifically, nn2.tar, which you can find [here](http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/A5%20Neural%20Networks.ipynb)). The way this code is set up, however, was more directed towards classification problems. You would provide a set of training and testing data, then provide unseen data to test the network's predictions based on its training. Our original thought was we could provide thousands of images of the first level, with the corresponding "best action" given that state. So, we would associate "jump up to the right" with this image: ![title](jump.jpg) 

as it leads to the best next state. There are two main problems with this idea, however:

1. *This is a ton of data to process and label.*

> Practically every possible frame and position of Mario / situations he encounters would have to be included in the testing data. For the image above, we would want Mario to jump up to the right, because it is a safer and essentially equally fast route. In other similar situations, the better option may be to duck, challenge the enemy, or take the damage (if it has the health or powerups to do so). There would have to be enough data provided for training so the network could make the appropriate decision.

2. *This approach doesn't exactly embody the purpose of this project.*

> We set out to have artificial intelligence solve the first level relatively unguided by ourselves; by providing the optimal moves for thousands of states, this felt far more similar to symbolic AI or hardcoding. There are tons of videos and experiments already conducted with "Tool-Assisted Speedruns" (TAS), where each movement is hardcoded. We wanted a more traditional AI approach.

So, we needed to find a different "flavor" of artificial intelligence that was better suited for our intentions. After some research, we concluded that some form of a genetic algorithm could have potential.  

## Genetic Algorithms ##

Genetic algorithms were introduced in 1960 by [John Holland](https://becominghuman.ai/a-peek-into-genetic-algorithms-6cf2d9a01b1a), and were inspired by the work of Charles Darwin. Genetic algorithms even copy much of the same terminology; generations consist of species, which are themselves comprised of genomes. They also rely on many of the same biological ideas as evolution: 

* `Selection`: Once per iteration (or "generation", as it is called in genetic algorithms) some portion of the current population is chosen to "breed" a new generation.  
* `Crossover`: Given two members of the population, some portion of their attributes are combined to generate a new member for the next generation.
* `Mutation`: The alternative to crossover; this allows for random change to occur, which could potentially lead to unexplored or undiscovered improvements.

Neuroevolutionary algorithms essentially make repeatedly semi-random input to output mappings, and determine the fitness of each possible solution. The fitter solutions are combined and mutated, so the next generation has some of the better aspects of the previous, while still changing in search of a more optimal solution. This picture shows the general idea:  ![title](GA.jpg)

While these types of algorithms have been around for a while, they have typically been overlooked for the more typical gradient descent or supervised learning algorithms. Interestingly, researchers at Uber (yes, Uber) have been performing research on neuroevolution, and have discovered that genetic algorithms and other AI applications that use neuroevolution are far more versatile than we had thought (the ScienceMag article can be found [here](https://www.sciencemag.org/news/2018/01/artificial-intelligence-can-evolve-solve-problems)). This and continuing hardware improvements in computers may allow for these algorithms to even beat industry-standard algorithms; at the minimum, they allow for a much wider application of AI.

Genetic algorithms do have a few drawbacks, however. For one, being an essentially brute-force approach to AI, it can take a significant amount of time as more and more complexity is added to the problem. Genetic algorithms do not typically perform well in a pass/fail environments, either, like is seen with classification problems. Another more complicated issue deals with the "hills and valleys" problem discussed in class; genetic algorithms (similarly to many other AI algorithms) tend to converge on local optima, not realizing that the optimal solution may involve non-optimal steps. Mutation can address this issue to some degree if implemented correctly. 

Despite these issues, we decided genetic algorithms (with possible modifications for the local optima problem) were the best way to approach this project. Due to the limited amount of time alloted for this project, we conceded that we may have been too ambitious with starting from the ground-up; so, we diverted slightly to replicating an existing project and discussing the structure, process, and results. The github with the code we used can be found [here](https://github.com/wert23239/Meta-MarIO/tree/master/Lua/NES).  

## Applying Genetic Algorithms to Mario ##

This approach to using AI to complete a Mario level has some distinct advantages. For one, we no longer have to provide an "answer key", as we do with supervised learning applications (our original thought). Instead, the network simply plays the level repeatedly, testing millions of different input and output combinations, learning by experience until it completes the level. Second, the way each generation, specie, and genome is evaluated (the "fitness" function) can be adapted to change how we value various actions. For example, we could place higher value on finishing with more time remaining, killing enemies, getting coins, or any combination of these very easily. This would require retraining, but the actual code changes are limited and easy to implement. 

The general idea of mapping inputs to outputs works remarkably well for Super Mario Bros., however. For this application, the inputs consist of Mario's current position; since we are using an emulator, we are able to get the exact position of the player at any given time. The output would be the corresponding action taken. These will be random at first, but as generations pass and the best species and genomes are crossbred, eventually we will end up with a neural network capable of solving the level. 

Similarly, we have an easily applicable fitness function; since we have a clear goal (finish the level), and are able to evaluate fitness depending on Mario's current position. As mentioned above, we can also easily add in additional factors. The Mario AI championship (see [here](http://www.marioai.org)) required the fitness function to be: 

> `S=D + 64df + 58dm + 58dgm + 42k+ 12kst + 17ksh + 4kf + 1024s + 32m + 24bh + 16c + 8t`

where `D` is the physical distance traveled by Mario from the start to his ﬁnal position; `df`, `dm` and `dgm` are the number of devoured ﬂowers, mushrooms and green mushrooms respectively; `k` is the number of killed enemies; `kst`, `ksh` and `kf` are the number of enemies killed by jumping, by throwing shells or by throwing ﬁreballs respectively; `s` is the ﬁnal status of the game, either won (1) or lost (0); `m` is the ﬁnal status of Mario, either small (0), big (1) or ﬁre (2); `bh` is the total number of hidden blocks found; `c` is the total number of coins collected and `t0` is the time left. Of course, given that this is more of an exploratory project, we simply defined fitness to be distance to the goal.  

For this application, genomes are the "hidden layer" of each specie; as pictured below, the genome would be the mapping between the current position of Mario and the corresponding buttons that should be pressed. Species are each granted a single life of Mario, and generations consist of species.  

![title](InputOutput.gif)

Above is a brief illustration showing the first steps in the learning process. Please note that this video is from another similar project, but done on a different Mario game. Details can be found [here](https://www.youtube.com/watch?v=qv6UVOQ0F44). The top left corner is what the network "sees"; based on the current block Mario is standing on (input), it will try various combinations of the eight possible buttons (outputs) leading to higher fitness values. As you can see, this training set took 71 species until it learned that sprinting to the right ('B' and 'Right' combined) led to the most fitness. The lines crossing towards the top right represent input and output connections. In a relatively untrained network like above, they seem relatively simple; by the end, however, they looked more like this:

![title](ComplicatedLayers.png)

These outputs shown above correspond to a singular position of Mario; obviously, the optimal outputs change drastically at each different state. These outputs came as a result of hundreds of the best species being crossbred and mutated, once per generation for a total of 35 generations. After these 35 generations, the fitness level surpassed the fitness threshold, meaning Mario had completed the level.

The genetic algorithms method does lead to some unfortunate complications, however. For one, a network trained on a set of levels is unlikely to solve ones that have drastic differences; for example, our trained network would be almost useless if applied to a water-only level. Some potential solutions to this issue involve increasing mutation when solution quality either drops or stagnates, or by introducing completely new species / genomes at random intervals. Another more complicated approach involving using another network for image recognition to identify its surroundings could also potentially resolve this issue; these were all a little beyond the scope of this project, however.

## Technical Aspects ##

Possibly the most tricky part of this project is working with the emulator; most emulators are unfortunately written with Windows operating systems in mind. [Bizhawk](http://tasvideos.org/Bizhawk.html), the best emulator we could find, was written in C# and, as such, does not naturally work on Linux. We attempted several hypervisors, but the way the emulator is coded relies on hardware and library packages specific to Windows operating systems, making it far more difficult than we had thought. As this utilizes a genetic algorithm, creating the network involves significantly more computations, and thus a GPU is preferable. Fortunately, Luke Burford was able to track down a Windows PC with a decent GPU. 

#### Inputs / Outputs ####

As mentioned above, inputs correspond directly to the position of Mario in the level. Fortunately, BizHawk makes retrieving this information very easy. Outputs correspond to button presses, or actions that Mario takes. There are eight buttons on a Nintendo Entertainment System, but only six are acceptable inputs for Super Mario Bros. {A, B, Up, Down, Right, Left}. As two buttons can be pressed at the same time, there are technically $2^6 = 64$ possible combinations. Of course, some of these combinations are impossible; it makes no sense for Mario to move left and right at the same time. So, by eliminating infeasible combinations, and accounting for the possibility of not pressing any buttons at all (staying still), we have $22$ combinations left. The image below illustrates these possible combinations:

![title](Buttons.png)

#### Mutation and Crossover ####

For each new generation, a new set of species must be created, largely based on the old species as described / shown above in the Genetic Algorithms section of this report. While the process itself is relatively simple, it does rely on programmer inputs for the percentages, or chance, that either occurs. In the code, this is defined with the following variables and values:

* `crossoverChance = 0.75`
* `nodeMutationChance = 0.5`
* `mutateConnectionsChance = 0.70`
* `disableMutationChance = 0.4`
* `enableMutationChance = 0.2`

CrossoverChance is relatively self-explanatory; based on the value of 0.75, there is a 75% chance each new specie is generated using crossover. Nodes within the network (essentially generated hidden layers) have a 50% chance to mutate. Input and output connections also have a 70% chance of experiencing mutation. Of course, mutation itself has a 40% chance of disabling mutation; if disabled, there is a 20% chance of reenabling it. So, this network has a pretty significant possibility for mutation worked in. This serves two purposes; one, it allows the network to explore previously unexplored combinations and possibilities, and two, it helps mitigate the hills and valleys problem. If a genome or generation converges to a local optimum, random mutation may help it explore different possibilities. 

We played around with several different combinations of values for these variables, with relatively surprising results. With lower mutation levels, the network actually seemed to achieve higher fitness faster; we imagine this is due to the first level of Super Mario Bros. being relatively straightforward, i.e. without any significant local maxima. Sprinting to the right is always the best option, while adding in a few jumps to survive and navigate simple obstacles. So, the network performs better sooner with more exploitation of the current optimal solutions than exploration of different outputs. If the mutation chance was too low, however, it struggled to attempt new actions, so the fitness rates dropped significantly. Changes to crossoverChance did not seem to have any consistent effects on fitness levels.

Of course, genetic algorithms don't apply mutation and crossover to the same consistent set of species; first, it eliminates the species with lower fitness levels. This is done once per generation, right before the mutation and crossbreeding occurs. The algorithm this code uses (`NeuralNetwork.lua` line 576) to determine which species should be removed is:
> `afs / (sum * p) `

where:
* `afs` is the average fitness score for the specie
* `sum` is the total average fitness for all species in the same generation
* `p` is the population, or number of species in the same generation

If the value of the equation is greater-than or equal to $1$, then the specie remains. If not, it is removed from the specie pool. So, in essense, this algorithm effectively determines whether each specie is performing above average, or below average compared to its peers. As ideally future generations are better than the previous, this step is very important in propogating species with higher fitness levels. It's also interesting to note that the number of species per generation is not hardcoded; it is possible (likely, even) that the number of species will decrease slightly as generations pass.


#### Stale Species and Timeout ####

![title](pipe.gif)

Above, you can see our training process for generation 4. By this point, the network is able to get Mario over the first pipe; however, the second pipe is taller, and requires holding the jump button down longer. As the network has not come across this more challenging obstacle yet, it has effectively stagnated. Occasionally it does jump high enough, but fails to recognize that it must also move to the right at the appropriate time. 

A more extreme example may be when the algorithm is just starting; as we discussed above, it can take in excess of 70 species before it even begins to move in the right direction. If species making no progress were allowed to run until they either died or completed the level, this network could take months to complete. Instead, species are "timed-out"; if they are not increasing fitness levels for a significant amount of time, they are killed, and another specie (hopefully more capable) is started.  

This brings up another interesting choice. If the specie above struggling with the taller pipe is timed-out, where should the next specie begin? Imagine the "best" specie so far is up next; it is competently able to navigate up to the taller pipe much faster than any other specie, but has not yet figured out how to get over it, either. If we simply place the new species at this obstacle, it will recieve unfairly low fitness score, likely causing it to be eliminated from the specie pool. So, we instead start every specie from the same point- the start of the level. This ensures fitness is scored fairly, and makes sure we do not accidentally remove competent species struggling with the same obstacle. This does mean that parts of the level may be rerun many, many times; to slightly counteract this, the level is scaled up to 800% speed (including Mario), and training is performed. 

Of course, there is always the chance that the same species fails to improve over subsequent generations. This code handles this possibility with the following constant:

`staleSpeciesConstant = 15`

Which means that after 15 generations, if a species has not improved, it is then removed from the species pool. This helps to further eliminate species that may be performing well enough to not be elimated, but are unable to navigate certain obstacles. With crossover likely to occur at the end of each generation, with an additional chance of mutation, we believe it is unlikely to continue for 15 generations (without being eliminated), without some measure of improvement. So, we left this constant alone; it is possible that lower values could speed up the learning process, at the potential cost of eliminating species with combinations that will be more valuable later on. In the video above, if we simply eliminated this specie because it fails to get over the larger pipe, we might miss out on a more optimal solution if it was to be crossbred with a species who was able to make it over the pipe.

#### Novelty Fitness ####

Of course, being a video game, the most direct path to the goal may not always be the optimal path. 

![title](noveltyFitness.jpg)

As an example, the level pictured in the image above (of a different Mario level, but same game - hence the poor graphics) would require Mario to move away from the goal at times in order to reach the end of the level. A fitness function could easily account for the vertical distance as well as the horizontal distance to the goal, but this doesn't change the fact that after the first drop, Mario will need to move farther away from the goal. 

If a specie is lucky enough to finish the entire second half of the level in relatively few tries, the increase of fitness at the end would be enough to keep it from being eliminated; this is extremely unlikely, however. So, we need a way to increase fitness based on exploring new paths. For this algorithm, the solution comes in the form of novelty fitness. Novelty fitness points are awarded to species that investigate previously unexplored areas, which provides some flexibility for species to explore new paths to the goal without being eliminated immediately. If the new path is eventually less optimal, then the specie will be removed by either stagnation or lower fitness levels (even with the novelty fitness). 

## The Code ##

As given in the introduction, the github used to replicate this project can be found [here](https://github.com/wert23239/Meta-MarIO/tree/master/Lua/NES). The code itself was written in Lua, which makes sense as it was a popular AI language before Python became the new standard. It also has the benefit of interfacing extremely well with the BizHawk emulator, which was one of the major issues for us trying to start on this project from scratch. If you are curious how the code interfaces with the emulator, more information can be found [here](http://datacrystal.romhacking.net/wiki/Super_Mario_Bros.:RAM_map). The 9 files we used to replicate this file are as follows:

`Constants.lua`
> This is the file that contains all of the adjustable constants, like those mentioned above in technical aspects. Changing these values can lead to more effective training, as we saw when we lowered the mutation rate. The wrong changes, such as setting `disableMutationChance` to 1.0, could lead to a network that never learns to solve the level at all. This is also where the possible buttons are defined, and therefore input and output sizes. `client.speedmode(800)` is responsible for the 800% speedup, as mentioned above under 'Stale Species and Timeout'.

`Fitness.lua`
> This file contains helper functions that assist in calculating the average fitness of a specie and generation. It is also responsible for ranking the highest species; in general, these functions are only used when determining which weak species to remove. It is helpful in seeing the progress of each generation in a numerical way, however!

`Forms.lua`
> This file is responsible for generating a window allowing the programmer to make simple modifications as the network is training. It also provides information on generational and specie statistics. Essentially a GUI for the program.

`Inputs.lua`
> This is where the main interactions with the BizHawk emulator occurs. It provides functions used to acquire Mario's position at any given time and mark enemy locations (if on screen). 

`Main.lua`
> As you may have imagined, this file is essentially the driver for the entire program. It is responsible for loading previous generational files (if provided), initializing the network, generations, species, and genomes, and begins the training process. It is also responsible for the timeout functionality mentioned above under 'Stale Species and Timeout'. It keeps track of which specie is currently active, and when it dies or is timed out, starts the next species while updating the previous specie's statistics.

`Mutations.lua`
> All mutation of genomes and species occurs here. Utilizing the constants defined in `Constants.lua`, it allows for some measure of randomness in the training process, which greatly helps minimize the local maxima issues. 

`NeuralNetwork.lua`
> This is where the majority of the work happens. In this file, the neural network is trained using a genetic algorithms approach. It is responsible for performing training, calling the appropriate mutation and crossover functions, restructing the specie pool (removing poorly performing species), and setting up / tracking generations.

`Pools.lua`
> This program utilizes ".pool" (essentially text) files in order to save the network at various states; this allows training to be paused and resumed at later times. All of this functionality is found in this file.

`Timeout.lua`
> This file consists of helper functions that allow `Main.lua` to determine when a specie should be timed-out.

## Results ##

All said and done, this network took 27 generations to complete the first level of Super Mario Bros. This was slightly faster than the 33 generations it took the programmers who designed the code; we believe this is due to lower mutation levels, which increased our fitness numbers much faster. The chart below shows the average fitness for approximately every five generations: 

![title](fitnessByGeneration.png)

Generation 22 was included as an example that while the majority of successive generations improved on the previous generation, the fitness levels did not strictly increase. Due to the randomness of crossover and mutation, it is possible for a new generation to actually score lower than the previous, as with generation 22.

Below are a few species from generation ten; they are finally becoming slightly more adept at navigating obstacles and enemies, but can still struggle at times:

![title](gen10.gif)

As you can see, Mario has now learned how to avoid enemies (even if it means moving away from the goal), and navigate the tougher obstacles, such as the increasingly taller pipes, albeit not without a decent amount of hesitation.

Here is the same segment of track for several species in generation twelve:

![title](gen15.gif)

In the span of just two generations, Mario seems consistently more confident in navigating obstacles, and has recognized that he can kill enemies by jumping on them. While this isn't factored into a higher fitness score, it is certainly a better option than jumping backwards to avoid the enemies completely (as seen in generation 10). 

And, finally, here is the complete run! Please note this is still operating at 800% speed.

![title](finalRun.gif)

## Conclusion ##

So, while we were unable to accomplish our original goal of starting from scratch, this project was a fantastic way to visualize the learning process a neural network goes through. While most neural networks can only be "tested" for accuracy at various points in the training process, we were able to supervise each step. It was also fascinating diving a little deeper into genetic algorithms and seeing how they were applied. 

Of course, we still have a few loose ends to tie up; for work distribution, Luke Burford was responsible for the majority of supervising the training process (he had the Windows desktop used for training), while Drew Boston performed the research into genetic algorithms. Both of us helped set up the necessary code to perform the training, and equally contributed to the report. 

To replicate our project, simply navigate to the releases section of the github linked above (or use this [link](https://github.com/wert23239/Super-Meta-MarIO/releases)) and follow the instructions in "Getting_Started.txt".

Thanks for a great semester!

In [3]:
import io
from IPython.nbformat import current
import glob
nbfile = glob.glob('final.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file final.ipynb is 4092
